In [1]:
import requests
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import osm2geojson  # pip install osm2geojson

# Define bounding box
north, south = 43.40011413701851, 43.15154808326906
west, east = 5.228894338539561, 5.565061518831354

# Overpass query: buildings as ways and relations
overpass_url = "https://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:180];
(
  way["building"]({south},{west},{north},{east});
  relation["building"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

# Request OSM data
print("⏳ Downloading OSM data for Marseille...")
response = requests.get(overpass_url, params={"data": query})
data = response.json()

# Convert to GeoJSON
print("🔄 Converting to GeoJSON...")
geojson = osm2geojson.json2geojson(data)

# Load into GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson["features"])
gdf.set_crs("EPSG:4326", inplace=True)

# Clean geometries
print("🧼 Fixing invalid geometries...")
gdf["geometry"] = gdf["geometry"].buffer(0)

# Merge all geometries into blocks
print("🔗 Merging all building shapes...")
merged = unary_union(gdf["geometry"])

# Create final GeoDataFrame
if isinstance(merged, MultiPolygon):
    merged_gdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=gdf.crs)
else:
    merged_gdf = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)

# Save result
output_file = "marseille_buildings_merged.gpkg"
merged_gdf.to_file(output_file, driver="GPKG")

print(f"✅ Done! Merged {len(gdf)} buildings into {len(merged_gdf)} shapes.")
print(f"📦 Saved to: {output_file}")


⏳ Downloading OSM data for Marseille...
🔄 Converting to GeoJSON...
🧼 Fixing invalid geometries...
🔗 Merging all building shapes...
✅ Done! Merged 366129 buildings into 135819 shapes.
📦 Saved to: marseille_buildings_merged.gpkg
